In [9]:
!pip install transformers torch torchvision einops timm peft sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.8 MB/s eta 0:00:00


In [10]:
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer

import pandas as pd

In [11]:
# Set up the model and tokenizer
model_path = 'h2oai/h2ovl-mississippi-800m'
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
#config.llm_config._attn_implementation = 'flash_attention_2'
config.vision_config.use_flash_attn = False
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    config=config,
    low_cpu_mem_usage=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens=2048, do_sample=True)

image_process.py:   0%|          | 0.00/6.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/h2oai/h2ovl-mississippi-800m:
- image_process.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_intern_vit.py:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/h2oai/h2ovl-mississippi-800m:
- modeling_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


conversation.py:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [12]:
!ls images

receipt10.jpg  receipt2.jpeg  receipt4.jpeg  receipt6.jpeg  receipt8.jpg
receipt1.png   receipt3.jpeg  receipt5.jpg   receipt7.png   receipt9.jpeg


In [13]:
path = 'images/'

images =['receipt1.png','receipt2.jpeg','receipt3.jpeg','receipt4.jpeg','receipt5.jpg',
'receipt6.jpeg','receipt7.png','receipt8.jpg','receipt9.jpeg','receipt10.jpg']

In [26]:
%%time

#question = "What is this image about. Give a short explanation."
question = "<image>\nRead the printed text in the image. What is this image about. Short answer:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_1 = question
answer_1 = answer
print('----- DONE -----')

receipt1.png  :  The image is a screenshot of an online shopping website for an e-commerce platform called "BIG! BIG!". The screenshot shows an advertisement for IspMoney cards, which are offered at a discounted price of 500 yen (51p = 10yen). The discounted price is accompanied by the text "51P 51P" as a promotional offer. One of the features advertised in the advertisement is "1840 izp money 梱" (which translates to "1840 isp money card) and "1840 izp money 暦" (which translates to "1840 high quality plastic card"). Unfortunately, I don't know the exact product and promotional pricing that is being advertised in the image.
receipt2.jpeg  :  The printed text in the image appears to be a list of addresses. The first address includes the name "LAWSON", the first part of an address number, "3-13", and a telephone number. The remaining text includes various symbols and numbers, such as "毒損", "トラウラン", and an asterisk and exclamation mark "*は一貫稼ししました" which translates to "was a forced laborer

In [17]:
%%time

#question = "<image>\nRead the printed text in the image. Answer this image is 'Business Card', 'Resume' or 'Recipt'. Also answer its reason. \nType: \nReason: "
question = "<image>\nExtract the type of the image, categorizing it as 'Business Card', 'Resume' or 'Recipt'. Type:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_2 = question
answer_2 = answer
print('----- DONE -----')

receipt1.png  :  Business Card
receipt2.jpeg  :  Business Card
receipt3.jpeg  :  The image is a receipt.
receipt4.jpeg  :  The type of the image is 'Business Card' based on the structured and professional nature of the details, including the Japanese writing and layout, the currency (Japanese yen), and the date. It does not contain typical elements of a resume or a receipt.
receipt5.jpg  :  Business Card.
receipt6.jpeg  :  Resumé
receipt7.png  :  Recipt.
receipt8.jpg  :  Business Card
receipt9.jpeg  :  Business Card
receipt10.jpg  :  Business Card
----- DONE -----
CPU times: user 23.7 s, sys: 45.4 ms, total: 23.7 s
Wall time: 25 s


In [18]:
%%time

question = "<image>\nRead the printed text in the image. Is this image a recipt? Yes or No:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_3 = question
answer_3 = answer
print('----- DONE -----')

receipt1.png  :  Yes.
receipt2.jpeg  :  Yes.
receipt3.jpeg  :  No.
receipt4.jpeg  :  Yes.
receipt5.jpg  :  No.
receipt6.jpeg  :  Yes.
receipt7.png  :  Yes.
receipt8.jpg  :  Yes
receipt9.jpeg  :  Yes.
receipt10.jpg  :  Yes
----- DONE -----
CPU times: user 24.2 s, sys: 31.5 ms, total: 24.2 s
Wall time: 24.3 s


In [27]:
%%time

question = "<image>\nRead the printed text in the image. How much is the total amount in this recipt? Amount:"

answer = []
for img in images:
  image_file = path+img
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(img , ' : ', response)
  answer.append(response)

question_4 = question
answer_4 = answer
print('----- DONE -----')

receipt1.png  :  5P 51P
receipt2.jpeg  :  5041.00
receipt3.jpeg  :  187
receipt4.jpeg  :  ¥3,340
receipt5.jpg  :  [A lotus flower, 128] 11, 109
receipt6.jpeg  :  24.20
receipt7.png  :  $45.00
receipt8.jpg  :  63.10
receipt9.jpeg  :  27.81
receipt10.jpg  :  53.28
----- DONE -----
CPU times: user 24 s, sys: 26.6 ms, total: 24.1 s
Wall time: 24.4 s


In [28]:
question_1, question_2, question_3, question_4

('<image>\nRead the printed text in the image. What is this image about. Short answer:',
 "<image>\nExtract the type of the image, categorizing it as 'Business Card', 'Resume' or 'Recipt'. Type:",
 '<image>\nRead the printed text in the image. Is this image a recipt? Yes or No:',
 '<image>\nRead the printed text in the image. How much is the total amount in this recipt? Amount:')

In [29]:
len(answer_1), len(answer_2), len(answer_3), len(answer_4)

(10, 10, 10, 10)

In [30]:
df = pd.DataFrame({'image':images,
                   question_1:answer_1,
                   question_2:answer_2,
                   question_3:answer_3,
                   question_4:answer_4})
df

,image,<image>\nRead the printed text in the image. What is this image about. Short answer:,"<image>\nExtract the type of the image, categorizing it as 'Business Card', 'Resume' or 'Recipt'. Type:",<image>\nRead the printed text in the image. Is this image a recipt? Yes or No:,<image>\nRead the printed text in the image. How much is the total amount in this recipt? Amount:
0,receipt1.png,The image is a screenshot of an online shoppin...,Business Card,Yes.,5P 51P
1,receipt2.jpeg,The printed text in the image appears to be a ...,Business Card,Yes.,5041.00
2,receipt3.jpeg,The image shows a check being paid with Japane...,The image is a receipt.,No.,187
3,receipt4.jpeg,The image shows a restaurant receipt from a Ja...,The type of the image is 'Business Card' based...,Yes.,"¥3,340"
4,receipt5.jpg,This image is a page from a document where Jap...,Business Card.,No.,"[A lotus flower, 128] 11, 109"
5,receipt6.jpeg,"This image contains a table with various date,...",Resumé,Yes.,24.20
6,receipt7.png,The text in the image appears to be a menu fro...,Recipt.,Yes.,$45.00
7,receipt8.jpg,This image contains an expired receipt from a ...,Business Card,Yes,63.10
8,receipt9.jpeg,This image depicts a receipt from a grocery st...,Business Card,Yes.,27.81
9,receipt10.jpg,A hand holding a receipt that lists the total ...,Business Card,Yes,53.28


In [33]:
answer_1

['The image is a screenshot of an online shopping website for an e-commerce platform called "BIG! BIG!". The screenshot shows an advertisement for IspMoney cards, which are offered at a discounted price of 500 yen (51p = 10yen). The discounted price is accompanied by the text "51P 51P" as a promotional offer. One of the features advertised in the advertisement is "1840 izp money 梱" (which translates to "1840 isp money card) and "1840 izp money 暦" (which translates to "1840 high quality plastic card"). Unfortunately, I don\'t know the exact product and promotional pricing that is being advertised in the image.',
 'The printed text in the image appears to be a list of addresses. The first address includes the name "LAWSON", the first part of an address number, "3-13", and a telephone number. The remaining text includes various symbols and numbers, such as "毒損", "トラウラン", and an asterisk and exclamation mark "*は一貫稼ししました" which translates to "was a forced laborer" with some other text at th

In [31]:
#model_path.split('/')[1]

In [34]:
df.to_csv(model_path.split('/')[1]+'.csv', index=False, encoding='sjis', errors='ignore')